In [16]:
import pandas as pd
import numpy as np
from functools import reduce

## <center> Reading in Data </center>
<center> We've collected our data from the below sources: </center>
<br>

<center> <a href="https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html" target="_blank"> State Population Data</a> </center>
<br>
<center> <a href="http://www.usa.com/rank/us--average-education-index--state-rank.htm" target="_blank">State Educational Ranking Data</a> </center>
<br>

<center> <a href="https://ssti.org/blog/useful-stats-capita-personal-income-state-2010-2015" target="_blank">State Income Data</a> </center>
<br>
<center> <a href="https://wonder.cdc.gov/ucd-icd10.html" target="_blank">Crude Death Rate where Drug Overdose is the Cause of Death Data</a> </center>





In [17]:
state_income_data_url = 'https://raw.githubusercontent.com/wafiakmal/Turquoise-Team-Data-Analysis-DS-Salaries/main/00_Raw_Data/pds/state%20income%202010.csv'
state_income = pd.read_csv(state_income_data_url)

state_educ_url = 'https://raw.githubusercontent.com/wafiakmal/Turquoise-Team-Data-Analysis-DS-Salaries/main/00_Raw_Data/pds/states%20education%20rank%202010%20thru%202014.csv'
state_educ = pd.read_csv(state_educ_url)

policy_url = 'https://raw.githubusercontent.com/wafiakmal/Turquoise-Team-Data-Analysis-DS-Salaries/main/00_Raw_Data/pds/States%20with%20missing%20policys.csv'
missing_policy = pd.read_csv(policy_url)

state_pop_url = 'https://raw.githubusercontent.com/wafiakmal/Turquoise-Team-Data-Analysis-DS-Salaries/main/00_Raw_Data/pds/state%20population%202010.csv'
state_pop = pd.read_csv(state_pop_url)

state_crud_death_url = 'https://raw.githubusercontent.com/wafiakmal/Turquoise-Team-Data-Analysis-DS-Salaries/main/00_Raw_Data/pds/Underlying%20Cause%20of%20Death%2C%201999-2020.csv'
state_crud_death = pd.read_csv(state_crud_death_url)





## Merging data where the matching key is State

In [18]:
to_merge_data_frames = [state_income, state_educ, missing_policy, state_pop, state_crud_death]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['State'],
                                            how='outer'), to_merge_data_frames)

## Dropping NA to get rid of NA row and District of Columbia

In [19]:
df_merged = df_merged.dropna()

## Creating columns to calculate averages
Per state (Florida, Texas, Washington), we're comparing their educational ranking, population from 2010, crude death rate (where cause of death is related to a drug overdose), and income. Below is an example.  
<center> income(state_x)/income(Florida) </center>
<br>
We're comparing these demographics to our three states to ensure that the chosen states are similar and proper for comparison.
<br>
<br>
We're also creating a flag column to evaluate each state's policy on the opioid crisis, which is based on Table 1 from: 
<br>
<br>
<center> <em> How States Are Tackling the Opioid Crisis: </em> </center>
<center> <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5349480/" target="_blank">Read paper here</a> </center>
<br>
<p> This paper is from 2017, however we're comfortable using this data as we're counting which policies are missing in each state. Therefore, if the policy is missing in 2017, then we can reasonably assume that the policy was missing in 2010 as well.
 


In [20]:
for states in df_merged['State']:
    income_percent = df_merged['2010 average income']/38718.0
    df_merged['df_merged_income_fl'] = income_percent
    income_percent1 = df_merged['2010 average income']/42821.0
    df_merged['df_merged_income_wash'] = income_percent1
    income_percent2 = df_merged['2010 average income']/38282.0
    df_merged['df_merged_income_tx'] = income_percent2

    edu_percent = df_merged['Education Rank']/37.0
    df_merged['df_merged_edu_fl'] = edu_percent
    edu_percent1 = df_merged['Education Rank']/9.0
    df_merged['df_merged_edu_wash'] = edu_percent1
    edu_percent2 = df_merged['Education Rank']/43.0
    df_merged['df_merged_edu_tx'] = edu_percent2

    pop_percent = df_merged['2010 popula tion ']/18801310.0
    df_merged['df_merged_pop_fl'] = pop_percent
    pop_percent1 = df_merged['2010 popula tion ']/6724540.0
    df_merged['df_merged_pop_wash'] = pop_percent1
    pop_percent2 = df_merged['2010 popula tion ']/25145561.0
    df_merged['df_merged_pop_tx'] = pop_percent2

    crude_percent = df_merged['Crude Rate']/16.2701
    df_merged['df_merged_crude_fl'] = crude_percent
    crude_percent1 = df_merged['Crude Rate']/13.6069
    df_merged['df_merged_crude_wash'] = crude_percent1
    crude_percent2 = df_merged['Crude Rate']/9.5206
    df_merged['df_merged_crude_tx'] = crude_percent2

    policy_flag = df_merged['Sum of MISSING from Questionarre ']
    df_merged['df_merged_policy_flag_fl'] = policy_flag > 4.0
    policy_flag1 = df_merged['Sum of MISSING from Questionarre ']
    df_merged['df_merged_policy_flag_wash'] = policy_flag1 > 3.0
    policy_flag2 = df_merged['Sum of MISSING from Questionarre ']
    df_merged['df_merged_policy_flag_tx'] = policy_flag1 > 5.0

    




df_metrics = df_merged.copy()

## After calculating comparisons
After we create the columns that compare demographics to Flordia, Texas, and Washington, respectively, we compute an average of each comparison. We're using this as a measure of "closeness."

In [21]:
df_metrics['eval_metric_fl'] = df_metrics[["df_merged_income_fl", "df_merged_edu_fl", "df_merged_pop_fl", "df_merged_crude_fl"]].mean(axis=1)
df_metrics['eval_metric_tx'] = df_metrics[["df_merged_income_tx", "df_merged_edu_tx", "df_merged_pop_tx", "df_merged_crude_tx"]].mean(axis=1)
df_metrics['eval_metric_wash'] = df_metrics[["df_merged_income_wash", "df_merged_edu_wash", "df_merged_pop_wash", "df_merged_crude_wash"]].mean(axis=1)

## Creating bounds for "closeness" 
After averaging the comparisons, we are filtering such that states that are 20% +/- 100% of our initial states (Texas, Washington, Florida). 

In [22]:
def calc_new_col(row):
    if row['eval_metric_fl'] <= 1.2 and row['eval_metric_fl'] >= 0.8:
        return True
    else:
        return False
df_metrics['in_bounds_fl'] = df_metrics.apply(lambda row: calc_new_col(row), axis=1)


def calc_new_col_wash(row):
    if row['eval_metric_wash'] <= 1.2 and row['eval_metric_wash'] >= 0.8:
        return True
    else:
        return False

df_metrics['in_bounds_wash'] = df_metrics.apply(lambda row: calc_new_col_wash(row), axis=1)


def calc_new_col_tx(row):
    if row['eval_metric_tx'] <= 1.2 and row['eval_metric_tx'] >= 0.8:
        return True
    else:
        return False

df_metrics['in_bounds_tx'] = df_metrics.apply(lambda row: calc_new_col_tx(row), axis=1)





## Using our bounds and chosing states based on their implemented policies
After bounding our comparison metric, we're selecting states where the bound is satisfied and the quantity of missing policies is greated than our comparison states (Texas, Florida, Washington).

In [23]:
df_florida = df_metrics[((df_metrics['in_bounds_fl']==True) & (df_metrics['df_merged_policy_flag_fl'] == True))]
df_washington = df_metrics[((df_metrics['in_bounds_wash']==True) & (df_metrics['df_merged_policy_flag_wash'] == True))]
df_texas = df_metrics[((df_metrics['in_bounds_tx']==True) & (df_metrics['df_merged_policy_flag_tx'] == True))]

## Below are dataframes containing the states that will be used in our analysis for comparison.

### We're analyzing Flordia against: California, Nevada, New York, and Texas

In [24]:
df_florida



,State,2010 average income,Education Rank,Sum of MISSING from Questionarre,2010 popula tion,State Code,Deaths,Population,Crude Rate,df_merged_income_fl,...,df_merged_crude_tx,df_merged_policy_flag_fl,df_merged_policy_flag_wash,df_merged_policy_flag_tx,eval_metric_fl,eval_metric_tx,eval_metric_wash,in_bounds_fl,in_bounds_wash,in_bounds_tx
4,California,42411.0,35.0,7.0,37253956.0,6.0,4049,37253956,10.8686,1.095382,...,1.141588,True,True,True,1.172698,1.136233,2.804518,True,False,True
27,Nevada,36918.0,44.0,5.0,2700551.0,32.0,576,2700551,21.3290,0.953510,...,2.240300,True,True,False,0.899317,1.083831,1.930036,True,False,True
31,New York,49283.0,22.0,6.0,19378102.0,36.0,1554,19378102,8.0194,1.272870,...,0.842321,True,True,True,0.847759,0.852988,1.766603,True,False,True
42,Texas,38282.0,43.0,5.0,25145561.0,48.0,2394,25145561,9.5206,0.988739,...,1.000000,True,True,False,1.018374,1.000000,2.527710,True,False,True


### We're analyzing Washington against: Alaska, Hawaii, Iowa, Kansas, Maine, Massachusetts, Minnesota, Montana, Nebraska, North Dakota, Oregon, South Dakota, Virginia, and Wyoming

In [25]:
df_washington

,State,2010 average income,Education Rank,Sum of MISSING from Questionarre,2010 popula tion,State Code,Deaths,Population,Crude Rate,df_merged_income_fl,...,df_merged_crude_tx,df_merged_policy_flag_fl,df_merged_policy_flag_wash,df_merged_policy_flag_tx,eval_metric_fl,eval_metric_tx,eval_metric_wash,in_bounds_fl,in_bounds_wash,in_bounds_tx
1,Alaska,47773.0,15.0,10.0,710231.0,2.0,83,710231,11.6863,1.233871,...,1.227475,True,True,True,0.598830,0.713120,0.936695,False,True,False
10,Hawaii,41594.0,16.0,7.0,1360301.0,15.0,150,1360301,11.0270,1.074281,...,1.158225,True,True,True,0.564203,0.667733,0.940453,False,True,False
14,Iowa,38687.0,25.0,6.0,3046355.0,19.0,253,3046355,8.3050,0.999199,...,0.872319,True,True,True,0.586837,0.646361,1.186152,False,True,False
15,Kansas,39235.0,17.0,9.0,2853118.0,20.0,269,2853118,9.4283,1.013353,...,0.990305,True,True,True,0.551012,0.631003,0.980584,False,True,False
18,Maine,37102.0,23.0,6.0,1328361.0,23.0,135,1328361,10.1629,0.958262,...,1.067464,True,True,True,0.568793,0.656088,1.091608,False,True,False
20,Massachusetts,51643.0,2.0,9.0,6547629.0,25.0,737,6547629,11.2560,1.333824,...,1.182278,True,True,True,0.606988,0.709549,0.807290,False,True,False
22,Minnesota,42567.0,8.0,4.0,5303925.0,27.0,397,5303925,7.4850,1.099411,...,0.786190,False,True,False,0.514444,0.573774,0.805447,False,True,False
25,Montana,34737.0,14.0,8.0,989415.0,30.0,118,989415,11.9262,0.897180,...,1.252673,True,True,True,0.515299,0.631250,0.847597,False,True,False
26,Nebraska,40023.0,18.0,9.0,1826341.0,31.0,117,1826341,6.4063,1.033705,...,0.672888,True,True,True,0.502769,0.552400,0.919266,False,True,False
33,North Dakota,42964.0,20.0,9.0,672591.0,38.0,23,672591,3.4196,1.109665,...,0.359179,True,True,True,0.474039,0.493337,0.894224,False,True,False


### We're analyzing Texas against: Arkansas, California, Georgia, Missouri, New York, and Wyoming

In [26]:
df_texas

,State,2010 average income,Education Rank,Sum of MISSING from Questionarre,2010 popula tion,State Code,Deaths,Population,Crude Rate,df_merged_income_fl,...,df_merged_crude_tx,df_merged_policy_flag_fl,df_merged_policy_flag_wash,df_merged_policy_flag_tx,eval_metric_fl,eval_metric_tx,eval_metric_wash,in_bounds_fl,in_bounds_wash,in_bounds_tx
3,Arkansas,31991.0,48.0,7.0,2915918.0,5.0,350,2915918,12.0031,0.826257,...,1.260750,True,True,True,0.754096,0.832164,1.849044,False,False,True
4,California,42411.0,35.0,7.0,37253956.0,6.0,4049,37253956,10.8686,1.095382,...,1.141588,True,True,True,1.172698,1.136233,2.804518,True,False,True
9,Georgia,34487.0,38.0,6.0,9687653.0,13.0,1052,9687653,10.8592,0.890723,...,1.140600,True,True,True,0.775112,0.827613,1.816576,False,False,True
24,Missouri,36638.0,33.0,6.0,5988927.0,29.0,993,5988927,16.5806,0.946278,...,1.741550,True,True,True,0.793948,0.926054,1.657857,False,False,True
31,New York,49283.0,22.0,6.0,19378102.0,36.0,1554,19378102,8.0194,1.272870,...,0.842321,True,True,True,0.847759,0.852988,1.766603,True,False,True
49,Wyoming,44846.0,21.0,8.0,563626.0,56.0,85,563626,15.0809,1.158273,...,1.584028,True,True,True,0.670682,0.816570,1.143192,False,True,True
